In [2]:
import torch
import csv

class Parser():
    def __init__(self, model):
        self.model = model
        self.layers_info = []

    def parse(self):
        # Iterate over all the layers of the model
        for name, module in self.model.named_modules():
            if isinstance(module, torch.nn.Conv2d):
                layer_type = "Conv2D"
                input_channels = module.in_channels
                output_channels = module.out_channels
                kernel_size = module.kernel_size
                stride = module.stride
                padding = module.padding
                layer_info = [name, layer_type, input_channels, output_channels, kernel_size, stride, padding]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.Linear):
                layer_type = "Linear"
                input_channels = module.in_features
                output_channels = module.out_features
                layer_info = [name, layer_type, input_channels, output_channels]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.BatchNorm2d):
                layer_type = "BatchNorm2D"
                input_channels = module.num_features
                layer_info = [name, layer_type, input_channels]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.ReLU):
                layer_type = "ReLU"
                layer_info = [name, layer_type]
                self.layers_info.append(layer_info)

            elif isinstance(module, torch.nn.MaxPool2d) or isinstance(module, torch.nn.AvgPool2d):
                if isinstance(module, torch.nn.MaxPool2d):
                    layer_type = "MaxPool2D"
                else:
                    layer_type = "AvgPool2D"
                kernel_size = module.kernel_size
                stride = module.stride
                padding = module.padding
                layer_info = [name, layer_type, kernel_size, stride, padding]
                self.layers_info.append(layer_info)

    def write_to_csv(self, file_path):
        # Write the layer information to a CSV file
        with open(file_path, "w") as f:
            writer = csv.writer(f)
            writer.writerow(["Name", "Type", "Input Channels", "Output Channels", "Kernel Size", "Stride", "Padding"])
            writer.writerows(self.layers_info)
            
# # example of using the class
# model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)
# parser = Parser(model)
# parser.parse()
# parser.write_to_csv("l


In [3]:
import torchvision.models as models

# Instantiate the ResNet50 model
resnet50 = models.resnet50(pretrained=True)

# Instantiate the ModelParser class and pass in the model
parser = Parser(resnet50)

# Call the parse_model method to parse the model
parser.parse()

# Call the write_to_csv method to write the parsed information to a CSV file
parser.write_to_csv('resnet50_final_parsed.csv')


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [13]:
import torch
import torch.nn as nn
import pandas as pd

# Load the pre-trained SSD-Large model
model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math='fp16')

# Create a dataframe to store the information
df = pd.DataFrame(columns=["layer_name", "operation", "in_channels", "out_channels", "filter_size", "image_in", "image_out", "stride", "pad", "pool_type", "data_size"])

# Iterate through the layers in the model
for i, layer in enumerate(model.modules()):
    layer_name = str(layer)
    operation = str(layer.__class__.__name__)

    if isinstance(layer, nn.Conv2d):
        in_channels = layer.in_channels
        out_channels = layer.out_channels
        filter_size = (layer.kernel_size[0], layer.kernel_size[1])
        stride = (layer.stride[0], layer.stride[1])
        pad = (layer.padding[0], layer.padding[1])
    elif isinstance(layer, nn.BatchNorm2d):
        in_channels = layer.num_features
        out_channels = layer.num_features
    elif isinstance(layer, nn.ReLU):
        in_channels = None
        out_channels = None
        filter_size = None
        image_in = None
        image_out = None

    elif isinstance(layer, nn.MaxPool2d) or isinstance(layer, nn.AvgPool2d):
        in_channels = None
        out_channels = None
        pool_type = str(layer.__class__.__name__)
        filter_size = (layer.kernel_size, layer.kernel_size)
        stride = (layer.stride, layer.stride)
        pad = (layer.padding, layer.padding)
    else:
        in_channels = None
        out_channels = None
        filter_size = None
        image_in = None
        image_out = None
        stride = None
        pad= None
        pool_type = None
        data_size = None
    df = df.append(pd.Series([layer_name, operation, in_channels, out_channels, filter_size, image_in, image_out, stride, pad, pool_type, data_size], index=df.columns), ignore_index=True)

df.to_csv("ssd_large_model_final.csv", index=False)
print("CSV file created successfully!")


Using cache found in /root/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


CSV file created successfully!


In [14]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.8 MB/s eta 0:00:00


In [15]:
import torch
from transformers import BertForSequenceClassification, BertConfig
import pandas as pd

# Load the pre-trained BERT-768 model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Create a dataframe to store the information
df = pd.DataFrame(columns=["layer_name", "operation", "in_channels", "out_channels", "filter_size", "stride", "pad", "pool_type", "data_size"])

# Iterate through the layers in the model
for i, layer in enumerate(model.bert.encoder.layer):
    layer_name = str(layer)
    operation = str(layer.__class__.__name__)
    in_channels = layer.attention.self.query.in_features
    out_channels = layer.attention.self.query.out_features
    filter_size = (3,3)
    stride = None
    pad = None
    pool_type = None
    data_size = None
    df = df.append(pd.Series([layer_name, operation, in_channels, out_channels, filter_size, stride, pad, pool_type, data_size], index=df.columns), ignore_index=True)

df.to_csv("BERT_768_final_model_info.csv", index=False)
print("CSV file created successfully!")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

CSV file created successfully!


In [16]:
import torch
import torchvision
import pandas as pd

# Load RetinaNet model
model = torchvision.models.detection.retinanet.retinanet_resnet50_fpn(pretrained=True)
model.eval()

layer_infos = []
for name, module in model.named_modules():
    layer_info = {
        'layer_name': name,
        'type': type(module).__name__
    }
    layer_infos.append(layer_info)
def save_to_csv(layer_infos, filename):
    df = pd.DataFrame(layer_infos)
    df.to_csv(filename, index=False)
# Save layer information to a CSV file
filename = 'retinanet_layer_final.csv'
save_to_csv(layer_infos, filename)


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RetinaNet_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=RetinaNet_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
